In [60]:
from pathlib import Path
import pyrosetta
import pyrosetta_help as ph
from types import ModuleType
from IPython.display import display, HTML

from collections import Counter
prc: ModuleType = pyrosetta.rosetta.core
prp: ModuleType = pyrosetta.rosetta.protocols
pru: ModuleType = pyrosetta.rosetta.utility
prcc: ModuleType = pyrosetta.rosetta.core.conformation
pr_scoring: ModuleType = pyrosetta.rosetta.core.scoring
pr_rs: ModuleType = pyrosetta.rosetta.core.select.residue_selector


logger = ph.configure_logger()
pyrosetta.distributed.maybe_init(extra_options=ph.make_option_string(no_optH=False,
                                                                     ex1=None,
                                                                     ex2=None,
                                                                     # mute='all',
                                                                     ignore_unrecognized_res=True,
                                                                     load_PDB_components=False,
                                                                     ignore_waters=True)
                                 )
pyrosetta.rosetta.basic.options.set_boolean_option('run:ignore_zero_occupancy', False)
pyrosetta.rosetta.basic.options.set_boolean_option('in:auto_setup_metals', True)

In [61]:
pose = pyrosetta.pose_from_file('peptide-bound.pdb')

In [62]:
constraint_weight = 1
HarmonicFunc = pr_scoring.func.HarmonicFunc
AtomPairConstraint = pr_scoring.constraints.AtomPairConstraint


atoms = (ph.get_AtomID_from_pymol_line(pose, '/7DA6/A/A/THR`126/O '),
         ph.get_AtomID_from_pymol_line(pose, '/7DA6/B/C/ARG`0/NH1 ')
        )
con = AtomPairConstraint(*atoms, HarmonicFunc(x0_in=3.4, sd_in=0.2))
pose.add_constraint(con)


atoms = (ph.get_AtomID_from_pymol_line(pose, '/7DA6/A/A/GLU`88/OE1 '),
         ph.get_AtomID_from_pymol_line(pose, '/7DA6/B/C/LYS`2/NZ ')
        )
con = AtomPairConstraint(*atoms, HarmonicFunc(x0_in=3.6, sd_in=0.2))
pose.add_constraint(con)

atoms = (#ph.get_AtomID_from_pymol_line(pose, '/7DA6/B/C/PHE`-1/CZ '),
         ph.get_AtomID(pose, 'C', -1, 'CZ'),
         ph.get_AtomID_from_pymol_line(pose, '/7DA6/A/A/HIS`21/N ')
        )

con = AtomPairConstraint(*atoms, HarmonicFunc(x0_in=4.1, sd_in=0.2))
pose.add_constraint(con)

atoms = (
         ph.get_AtomID(pose, 'C', -1, 'CE2'),
         ph.get_AtomID_from_pymol_line(pose, '/7DA6/A/A/TYR`90/OH ')
        )

con = AtomPairConstraint(*atoms, HarmonicFunc(x0_in=3.6, sd_in=0.2))
pose.add_constraint(con)


atoms = (
         ph.get_AtomID_from_pymol_line(pose, '/x0310_template//A/HIS`21/NE2 '),
         ph.get_AtomID_from_pymol_line(pose, '/x0310_template//A/CYS`110/SG ')
        )

con = AtomPairConstraint(*atoms, HarmonicFunc(x0_in=3.9, sd_in=0.2))
pose.add_constraint(con)

In [63]:
movemap = pyrosetta.MoveMap()
chain_C: pru.vector1_bool = pr_rs.ChainSelector(2).apply(pose)
movemap.set_bb(chain_C)
movemap.set_chi(chain_C)

scorefxn: pr_scoring.ScoreFunction = pyrosetta.get_fa_scorefxn()
scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, constraint_weight)
scorefxn.set_weight(pr_scoring.ScoreType.angle_constraint, constraint_weight)
scorefxn.set_weight(pr_scoring.ScoreType.atom_pair_constraint, 5)
pyrosetta.rosetta.basic.options.set_boolean_option('relax:constrain_relax_to_start_coords', True)
pyrosetta.rosetta.basic.options.set_boolean_option('relax:coord_constrain_sidechains', True)
pyrosetta.rosetta.protocols.relax.FastRelax.register_options()
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 15)
relax.constrain_relax_to_start_coords(True)
relax.set_movemap(movemap)
relax.apply(pose)

In [64]:
pose.dump_pdb('test.pdb')

True

In [97]:
from rdkit import Chem
from rdkit.Chem import AllChem
from typing import Tuple

ligand: Chem.Mol = Chem.MolFromPDBBlock( ph.get_pdbstr( pose.split_by_chain(2) ), removeHs=True)
emol = Chem.RWMol(ligand)
emol.BeginBatchEdit()

for bond in ligand.GetBonds():
    fore: Chem.Atom = bond.GetBeginAtom()
    fore_name: str = fore.GetPDBResidueInfo().GetName().strip()
    aft: Chem.Atom = bond.GetEndAtom()
    aft_name: str = aft.GetPDBResidueInfo().GetName().strip()
    if 'C' in (fore_name, aft_name) and 'N' in (fore_name, aft_name):
        emol.RemoveBond( fore.GetIdx(), aft.GetIdx() )
emol.CommitBatchEdit()
residues: Tuple[Chem.Mol] = AllChem.GetMolFrags(emol.GetMol(), asMols=1)

In [98]:
from followup.prep_fragalysis import prep, generate_header
header = generate_header(method='Foo',
                         ref_url='https://www.example.com',
                         submitter_name='unknown',
                         submitter_email='a@b.c',
                         submitter_institution='Nowehere',
                         generation_date='2012-12-12',
                         smiles='CN1C=NC2=C1C(=O)N(C(=O)N2C)C',
                         extras={'∆∆G': 'W. Josiah Gibbs'})
prep(df, header, 'mol', 'id', 'foo.sdf', 'x1234', 'x1234', ['∆∆G'])

In [124]:
from gist_import import GistImporter
from types import ModuleType
import pandas as pd

df = pd.DataFrame([{'mol': res, 
                    'resi': f'{res.GetAtomWithIdx(0).GetPDBResidueInfo().GetResidueName()}{res.GetAtomWithIdx(0).GetPDBResidueInfo().GetResidueNumber()}',
                    'ref_mols': 'x0152_1A'
                   } 
                   for res in residues])
fu: ModuleType = GistImporter.from_github('https://raw.githubusercontent.com/matteoferla/Fragment-hit-follow-up-chemistry/main/followup/prep_fragalysis.py').to_module()

header: Chem.Mol = fu.generate_header(method='7DA6-ligand',
                         ref_url='https://www.rcsb.org/structure/7DA6',
                         submitter_name='PDB',
                         submitter_email='a@b.c',
                         submitter_institution='PDB',
                         smiles='CN1C=NC2=C1C(=O)N(C(=O)N2C)C', extras={})
fu.prep(header=header, df=df, mol_col='mol', name_col='resi', outfile='7DA6-fragmented.sdf', ref_pdb_name='x0310_0A')

[15:47:43] Molecule does not have explicit Hs. Consider calling AddHs()


In [119]:
df

,mol,resi
0,<rdkit.Chem.rdchem.Mol object at 0x17e2d4200>,PHE-1
1,<rdkit.Chem.rdchem.Mol object at 0x17e2d4f90>,ARG0
2,<rdkit.Chem.rdchem.Mol object at 0x17e2d5380>,GLY1
3,<rdkit.Chem.rdchem.Mol object at 0x17e2d53f0>,LYS2


In [110]:
Chem.AtomPDBResidueInfo.GetRe

rdkit.Chem.rdchem.AtomPDBResidueInfo